Karthik Konath (kk28699), Kyle Polansky (kpp446)

# <center>EE 379K-DS Lab 6<center>

# Problem 1

In this problem we will use synthetic data sets to explore the bias-variance tradeoff incurred by using regularization.

Generate data of the form: $y=Xβ+\epsilon$, where X is an n×p matrix where n= 51,p= 50, and each $X_{ij}$ ∼N(0,1). Also, generate the noise according to $\epsilon_i ∼N(0,1/4). Let β be the all ones vector (for simplicity).

By repeatedly doing this experiment and generating fresh data (fresh X, and y, and hence $\epsilon$ – but make that you’re not reseting your random seed!) but keeping β fixed, you will estimatemany different solutions,ˆβ.  Estimate the mean and variance of ˆβ. Note that ˆβ is a vector, so for this exercise simply estimate the variance of a single component.

Use ridge regression, i.e., l2 regularization. Vary the regularization coefficient λ= 0.01,0.1,1,10,100 and repeat the above experiment. What do you observe? As you increase λ is the model becoming more simple or more complex? As you increase λ is performance becoming better or worse? Also compute LOOCV for each λ. How does the value of LOOCV, and in particular how it changes as λ varies, compare with what you observe for the explicitly computed variance?

Read about the Bootstrap, and try to use it to compute the variance (as above), but with asingle copy of the data, rather than with many fresh copies of the data.

# Problem 2. Problem 9 from Chapter 6.

(Predicting  the  number  of  applications  in  College)  Note  that  you  will  have  to  read  about  PCR(Principal Components Regression) and PLS (Partial Least Squares ) in the book, since we did notdiscuss these in class.

 In this exercise, we will predict the number of application

(a) Split the data set into a training set and a test set.

In [86]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression

df = pd.read_csv("College.csv", header = 0, index_col = 0)
df.head()

df.replace({"Yes" : 1, "No" : 0}, inplace = True)
x = df.drop(["Apps"], axis = 1)
y = df.loc[:,"Apps"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

(b) Fit a linear model using least squares on the training set, and report the test error obtained.

In [87]:
fit = LinearRegression().fit(x_train, y_train)
y_pred = fit.predict(x_test)

print(np.sqrt(mean_squared_error(y_test, y_pred)))

1010.6636061663704


(c) Fit a ridge regression model on the training set, with λ chosen by cross-validation. Report the test error obtained.

In [88]:
alphas = np.arange(0.001,10,.01)
fit = RidgeCV(alphas = alphas).fit(x_train, y_train)
y_pred = fit.predict(x_test)

print(np.sqrt(mean_squared_error(y_test, y_pred)))

1012.8559499061809


(d) Fit a lasso model on the training set, with λ chosen by crossvalidation. Report the test error obtained, along with the number of non-zero coefficient estimates.

In [89]:
fit = LassoCV(alphas = alphas).fit(x_train, y_train)
y_pred = fit.predict(x_test)

print(np.sqrt(mean_squared_error(y_test, y_pred)))

print(sum(fit.coef_ > 0.1) + sum(fit.coef_ < -0.1)) #coefficients less than .1 considered 0.

1014.3092155895241
12


(e) Fit a PCR model on the training set, with M chosen by crossvalidation. Report the test error obtained, along with the value of M selected by cross-validation.

In [90]:
total_variance = 0

for i in range(1,18):
    pca = PCA(n_components = i, svd_solver = "full")
    score = -1 * cross_val_score(pca, x, y, cv = 5).mean()
    pca  = pca.fit(x_train, y_train)
    if total_variance < .9:
        total_variance += pca.explained_variance_ratio_[-1]
        m = i

fit = LinearRegression().fit(x_train.iloc[:,:m+1] , y_train)
x_test_pcr = x_test.iloc[:,:m+1]
y_pred = pcr.predict(x_test_pcr)

print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(m)

1046.6227708719919
3


(f) Fit a PLS model on the training set, with M chosen by crossvalidation. Report the test error obtained, along with the value of M selected by cross-validation.

In [91]:
m = 0
best_score = 0

for i in range(1,18):
    score = -1 * cross_val_score(PLSRegression(n_components = i), x, y, cv = 5).mean()
    if score < best_score:
        best_score = score
        m = i
    
fit = PLSRegression(n_components = m).fit(x_train, y_train)
y_pred = fit.predict(x_test)

print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(m)

3028.1645714290194
1


(g) Comment on the results obtained. How accurately can we predict the number of college applications received? Is there much difference among the test errors resulting from these five approaches?

All of the models are roughly similar, with the exception of PLS which is significantly worse than the other models. In our case, the linear model has the lest rmse value, very closely followed by ridge and lasso. Using the linear model, college applications received can be estimated with a RMSE of roughly 1000.

# Problem 3. Problem 11 from Chapter 6.

(Predicting crime in Boston)

We will now try to predict per capita crime rate in the Boston data set.

(a) Try out some of the regression methods explored in this chapter, such as best subset selection, the lasso, ridge regression, and PCR. Present and discuss results for the approaches that you consider.

(b) Propose a model (or set of models) that seem to perform well on this data set, and justify your answer. Make sure that you are evaluating model performance using validation set error, crossvalidation, or some other reasonable alternative, as opposed to using training error.

(c) Does your chosen model involve all of the features in the data set? Why or why not?

# Problem 4

This is a written problem, supporting Problem 9 above.  Note that a lot of this has been solved in class, but it is good for you to try to do it again without referencing the class notes.

Consider the Least Squares optimization problem, given data X and y ...

Note that $x_i$ represents the ith row of X and hence is a row-vector.  Hence $x_{iβ}$ represents the dot product between the p-length vectors $x_i$ and β. Derive a closed form solution (as wedid in class) for $ˆβ_{LS}$, by expanding out, taking the derivative and setting it equal to zero. It might be easiest to work in vector notation rather than deal with the individual $x_i$’s.

Now consider the Ridge Regression problem ... Use the same approach as above to again derive a closed form expression for the solution, $ˆβ_R$.